In [1]:
import hail as hl
import pandas as pd

Loading BokehJS ...

In [2]:
hl.init(default_reference = 'GRCh38',
                tmp_dir = "gs://wes-bipolar-tmp-4day/")

/opt/conda/miniconda3/lib/python3.10/site-packages/hail/context.py:347: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 60383
SPARKMONITOR_LISTENER: Application Started: application_1718122941138_0003 ...Start Time: 1718128940207


Running on Apache Spark version 3.3.2
SparkUI available at http://rye-m.c.wes-bipolar.internal:37523
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.128-eead8100a1c1
LOGGING: writing to /home/hail/hail-20240611-1802-0.2.128-eead8100a1c1.log


In [3]:
# Full manifest before sample-filtering 
MANIFEST = 'gs://2024-wgspd/files/2024_WGSPD_merged-manifest.tsv'
manifest = hl.import_table(MANIFEST, delimiter='\t',
                          key = "s", impute = True)
manifest.describe()
manifest.count()

2024-06-11 18:02:41.681 Hail: INFO: Reading table to impute column types 1) / 1]
2024-06-11 18:02:43.544 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'sex_new' as type str (imputed)
  Loading field 'sex_old' as type str (imputed)
  Loading field 'SEX' as type str (imputed)
  Loading field 'primary_disease_new' as type str (imputed)
  Loading field 'primary_disease_new_fixed' as type str (imputed)
  Loading field 'primary_disease_old' as type str (imputed)
  Loading field 'primary_disease_old_fixed' as type str (imputed)
  Loading field 'PRIMARY_DISEASE' as type str (imputed)
  Loading field 'CASECON' as type str (imputed)


----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'sex_new': str 
    'sex_old': str 
    'SEX': str 
    'primary_disease_new': str 
    'primary_disease_new_fixed': str 
    'primary_disease_old': str 
    'primary_disease_old_fixed': str 
    'PRIMARY_DISEASE': str 
    'CASECON': str 
----------------------------------------
Key: ['s']
----------------------------------------


35527

In [4]:
# Subset dense MT before variant-filtering (by gnomad)
MT = 'gs://gnomad-subsets-2024/gnomad-v3/202403/20240328_subset_dense-callstats.mt'
mt = hl.read_matrix_table(MT)
mt.count()

(588713326, 35527)

In [5]:
# Annotate with phenotype info
mt = mt.annotate_cols(pheno = manifest[mt.s].CASECON)
#mt = mt.annotate_cols(primary_disease = manifest[mt.s].PRIMARY_DISEASE)

### Sites of interest

In [32]:
#f = mt.filter_rows(hl.is_defined(variant_ht[mt.locus, mt.alleles]))
f = hl.filter_intervals(mt, [hl.parse_locus_interval('chr12:120291700-120292000')])
f = f.checkpoint("gs://2024-wgspd/NDD_RNU4-2/20240610_NDD_RNU4-2_SNV_gene-range.mt", overwrite = True)

2024-06-11 17:16:32.669 Hail: INFO: wrote matrix table with 366 rows and 35527 columns in 1 partition to gs://2024-wgspd/NDD_RNU4-2/20240610_NDD_RNU4-2_SNV_gene-range.mt


In [33]:
f = hl.read_matrix_table("gs://2024-wgspd/NDD_RNU4-2/20240610_NDD_RNU4-2_SNV_gene-range.mt")
f.describe()
f.count()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'pheno': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'a_index': int32
    'was_split': bool
    'AC_raw': int32
    'AN_raw': int32
    'AF_raw': float32
    'AC': int32
    'AN': int32
    'AF': float32
----------------------------------------
Entry fields:
    'GT': call
    'DP': int32
    'GQ': int32
    'MIN_DP': int32
    'RGQ': int32
    'END': int32
    'PID': str
    'SB': array<int32>
    'gvcf_info': struct {
        ClippingRankSum: float64, 
        BaseQRankSum: float64, 
        MQ: float64, 
        MQRankSum: float64, 
        MQ_DP: int32, 
        QUALapprox: int32, 
        RAW_MQ: float64, 
        ReadPosRankSum: float64, 
        VarDP: int32
    }
    'PGT': call
    'AD': array<int32>
    'PL': array<int32>
    'adj': bool
------------------

(366, 35527)

In [34]:
f = f.filter_cols(f.pheno != "OTHER")
f = f.annotate_entries(non_ref = f.GT.is_non_ref())
f.count()

(366, 32252)

In [20]:
manifest.aggregate(hl.agg.counter(manifest.CASECON))

{'CASE': 9213, 'CTRL': 23039, 'OTHER': 3275}

In [35]:
f_case_con = f.annotate_rows(case_non_ref = hl.agg.count_where(f.non_ref & (f.pheno == "CASE")),
                             con_non_ref = hl.agg.count_where(f.non_ref & (f.pheno == "CTRL")),
                             #case_ref = hl.agg.count_where(~f.non_ref & (f.pheno == "CASE")),
                             #con_ref = hl.agg.count_where(~f.non_ref & (f.pheno == "CTRL")),
                             )
f_case_con = f_case_con.annotate_rows(case_ref = 9213 - f_case_con.case_non_ref,
                                      con_ref = 23039 - f_case_con.con_non_ref
                             )
f_case_con = f_case_con.annotate_rows(fisher = hl.expr.functions.fisher_exact_test(
    hl.int(f_case_con.case_non_ref),
    hl.int(f_case_con.case_ref),
    hl.int(f_case_con.con_non_ref),
    hl.int(f_case_con.con_ref)))
h = f_case_con.rows()
h = h.order_by(h.fisher.p_value)
h.show(n = 366)

2024-06-11 17:17:52.183 Hail: INFO: Ordering unsorted dataset with network shuffle


+-----------------+-------------------------------------+------+---------+
| locus           | alleles                             | rsid | a_index |
+-----------------+-------------------------------------+------+---------+
| locus<GRCh38>   | array<str>                          | str  |   int32 |
+-----------------+-------------------------------------+------+---------+
| chr12:120291710 | ["C","G"]                           | NA   |       1 |
| chr12:120291716 | ["T","A"]                           | NA   |       1 |
| chr12:120291744 | ["T","TA"]                          | NA   |       2 |
| chr12:120291909 | ["G","A"]                           | NA   |       1 |
| chr12:120291914 | ["T","C"]                           | NA   |       1 |
| chr12:120291727 | ["C","A"]                           | NA   |       1 |
| chr12:120291905 | ["G","A"]                           | NA   |       1 |
| chr12:120291824 | ["T","G"]                           | NA   |       2 |
| chr12:120291915 | ["G","A"]                           | NA   |       1 |
| chr12:120291878 | ["C","G"]                           | NA   |       1 |
| chr12:120291726 | ["T","TC"]                          | NA   |       4 |
| chr12:120291912 | ["T","G"]                           | NA   |       2 |
| chr12:120291719 | ["T","C"]                           | NA   |       2 |
| chr12:120291953 | ["A","T"]                           | NA   |       3 |
| chr12:120291865 | ["T","A"]                           | NA   |       1 |
| chr12:120291875 | ["T","C"]                           | NA   |       1 |
| chr12:120291740 | ["T","A"]                           | NA   |       1 |
| chr12:120291726 | ["T","A"]                           | NA   |       1 |
| chr12:120291789 | ["C","T"]                           | NA   |       2 |
| chr12:120291970 | ["C","G"]                           | NA   |       2 |
| chr12:120291904 | ["G","A"]                           | NA   |       1 |
| chr12:120291906 | ["A","AT"]                          | NA   |       1 |
| chr12:120291817 | ["G","A"]                           | NA   |       1 |
| chr12:120291976 | ["G","C"]                           | NA   |       1 |
| chr12:120291706 | ["C","A"]                           | NA   |       1 |
| chr12:120291751 | ["TTAAA","T"]                       | NA   |       2 |
| chr12:120291757 | ["A","C"]                           | NA   |       1 |
| chr12:120291773 | ["G","A"]                           | NA   |       1 |
| chr12:120291806 | ["C","G"]                           | NA   |       1 |
| chr12:120291919 | ["G","C"]                           | NA   |       2 |
| chr12:120291964 | ["G","T"]                           | NA   |       1 |
| chr12:120291735 | ["C","T"]                           | NA   |       2 |
| chr12:120291876 | ["G","A"]                           | NA   |       1 |
| chr12:120291944 | ["A","C"]                           | NA   |       1 |
| chr12:120291747 | ["A","G"]                           | NA   |       1 |
| chr12:120291758 | ["T","G"]                           | NA   |       3 |
| chr12:120291870 | ["C","T"]                           | NA   |       1 |
| chr12:120291716 | ["T","C"]                           | NA   |       2 |
| chr12:120291751 | ["T","C"]                           | NA   |       1 |
| chr12:120291918 | ["C","T"]                           | NA   |       3 |
| chr12:120291758 | ["T","C"]                           | NA   |       2 |
| chr12:120291844 | ["T","C"]                           | NA   |       1 |
| chr12:120291752 | ["T","A"]                           | NA   |       1 |
| chr12:120291748 | ["C","T"]                           | NA   |       2 |
| chr12:120291748 | ["CTT","C"]                         | NA   |       1 |
| chr12:120291866 | ["A","G"]                           | NA   |       1 |
| chr12:120291878 | ["C","T"]                           | NA   |       2 |
| chr12:120291956 | ["T","G"]                           | NA   |       2 |
| chr12:120291907 | ["A",

In [36]:
h = h.flatten()
h.export("gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_gene-range_fisher.tsv", delimiter = "\t")

2024-06-11 17:19:35.188 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-06-11 17:20:26.222 Hail: INFO: merging 2 files totalling 48.8K... + 1) / 1]
2024-06-11 17:20:26.619 Hail: INFO: while writing:
    gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_gene-range_fisher.tsv
  merge time: 396.492ms


In [6]:
variant_df = pd.DataFrame({"locus" : ["chr12:120291878"], "alleles": [["C","G"]]})
variant_ht = hl.Table.from_pandas(variant_df)
variant_ht = variant_ht.annotate(locus = hl.parse_locus(variant_ht.locus))
variant_ht = variant_ht.key_by("locus", "alleles")
#variant_ht = variant_ht.key_by("locus")
variant_ht.show()

,
locus,alleles
locus<GRCh38>,array<str>
chr12:120291878,"[""C"",""G""]"


In [12]:
i = hl.filter_intervals(mt, [hl.parse_locus_interval('chr12:120291878-120291879')]).persist()
i = i.filter_rows(hl.is_defined(variant_ht[i.locus, i.alleles]))
i.show()

2024-06-11 18:06:06.568 Hail: INFO: wrote matrix table with 2 rows and 35527 columns in 1 partition to gs://wes-bipolar-tmp-4day/persist_MatrixTablerAEMYRtqjM
2024-06-11 18:06:11.513 Hail: INFO: Coerced sorted dataset


,
locus,alleles
locus<GRCh38>,array<str>
chr12:120291878,"[""C"",""G""]"


In [15]:
i = i.filter_cols(i.pheno != "OTHER")
i = i.annotate_entries(non_ref = i.GT.is_non_ref())
i = i.annotate_cols(carrier = hl.agg.count_where(i.non_ref))

In [18]:
i.aggregate_cols(hl.agg.counter(i.carrier))

2024-06-11 18:07:18.187 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2024-06-11 18:07:19.646 Hail: INFO: Coerced sorted dataset


{0: 32244, 1: 8}

In [20]:
ic = i.cols()
ic = ic.filter(ic.carrier == 1)
manifest[ic.s].show()

2024-06-11 18:08:39.616 Hail: INFO: Coerced sorted dataset
2024-06-11 18:08:41.632 Hail: INFO: Coerced sorted dataset
2024-06-11 18:08:42.243 Hail: INFO: Coerced sorted dataset


+--------------+----------------+----------------+------------+
| s            | <expr>.sex_new | <expr>.sex_old | <expr>.SEX |
+--------------+----------------+----------------+------------+
| str          | str            | str            | str        |
+--------------+----------------+----------------+------------+
| "8007540361" | "Male"         | "Male"         | "Male"     |
| "8007540388" | "Female"       | "Female"       | "Female"   |
| "8007540397" | "Female"       | "Female"       | "Female"   |
| "8007540454" | "Male"         | "Male"         | "Male"     |
| "8007540652" | "Female"       | "Female"       | "Female"   |
| "8007541280" | "Female"       | "Female"       | "Female"   |
| "8007541904" | "Male"         | "Male"         | "Male"     |
| "8007541952" | "Male"         | "Male"         | "Male"     |
+--------------+----------------+----------------+------------+

+----------------------------+----------------------------------+
| <expr>.primary_disease_new | <expr>.primary_disease_new_fixed |
+----------------------------+----------------------------------+
| str                        | str                              |
+----------------------------+----------------------------------+
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Bipolar"                  | "BD"                             |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
+----------------------------+----------------------------------+

+----------------------------+----------------------------------+
| <expr>.primary_disease_old | <expr>.primary_disease_old_fixed |
+----------------------------+----------------------------------+
| str                        | str                              |
+----------------------------+----------------------------------+
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Bipolar"                  | "BD"                             |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
| "Schizophrenia"            | "SCZ"                            |
+----------------------------+----------------------------------+

+------------------------+----------------+
| <expr>.PRIMARY_DISEASE | <expr>.CASECON |
+------------------------+----------------+
| str                    | str            |
+------------------------+----------------+
| "SCZ"                  | "CASE"         |
| "SCZ"                  | "CASE"         |
| "SCZ"                  | "CASE"         |
| "BD"                   | "CASE"         |
| "SCZ"                  | "CASE"         |
| "SCZ"                  | "CASE"         |
| "SCZ"                  | "CASE"         |
| "SCZ"                  | "CASE"         |
+------------------------+----------------+

In [22]:
ic.s.collect()

2024-06-11 18:09:04.537 Hail: INFO: Coerced sorted dataset


['8007540361',
 '8007540388',
 '8007540397',
 '8007540454',
 '8007540652',
 '8007541280',
 '8007541904',
 '8007541952']